In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/alvaroromangomez/Documents/Documentos personales/Formación/Master Bioinformática y Estadística/TFM/tfm_alvaro/Codigo/notebooks':
  os.chdir(r'/Users/alvaroromangomez/Documents/Documentos personales/Formación/Master Bioinformática y Estadística/TFM/tfm_alvaro/Codigo/notebooks')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/3s/vv1d0lmn7g134m4psncn2_q80000gn/T/ipykernel_13561/1023205354.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap.py": 1685228246.3990028, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap_external.py": 1685228246.4039905, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codecs.py": 1685228243.6488864, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/aliases.py": 1685228245.1487875, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/__init__.py": 1685228245.143839, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/utf_8.py": 1685228245.728661, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library

In [2]:
import numpy as np
import pandas as pd

import deepchem as dc
import tensorflow as tf
from deepchem.models.graph_models import GraphConvModel

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

In [3]:
input_path = "../data/processed/smiles/"

traing_path = input_path + "train_data/"
test_path = input_path + "test_data/"
output_path = ""

In [4]:
# FILES
molecules_train_file = "smiles_activity_training.csv"
molecules_test_file = "smiles_activity_test.csv"

In [5]:
molecules_train = pd.read_csv(input_path + molecules_train_file)
molecules_test = pd.read_csv(input_path + molecules_test_file)

In [6]:
smiles_train = molecules_train["canonical_smiles"]
smiles_test = molecules_test["canonical_smiles"]

In [7]:
activity_train = molecules_train["activity"]
activity_test = molecules_test["activity"]

In [8]:
featurizer = dc.feat.ConvMolFeaturizer()

In [9]:
molecules_graphs_train = featurizer.featurize(smiles_train)
molecules_graphs_test = featurizer.featurize(smiles_test)

In [10]:
train_dataset = dc.data.NumpyDataset(X=molecules_graphs_train, y=activity_train)
test_dataset = dc.data.NumpyDataset(X=molecules_graphs_test, y=activity_test)


In [11]:
def objective(trial):
    dropout = trial.suggest_float("dropout", 0.2, 0.5, step=0.1)
    graph_conv_layers = trial.suggest_categorical(
        "graph_conv_layers",
        [
            [128, 128, 64, 32, 32, 16, 16],
            [128, 128, 64, 32, 32, 16, 8, 8, 4],
            [256, 128, 64, 32, 16],
            [64, 64, 64, 64, 64, 64, 64],
        ],
    )
    dense_layer_size = trial.suggest_categorical(
        "dense_layer_size", [200, 128, 64, 32, 16]
    )
    batch_size = trial.suggest_categorical("batch_size", [64, 32, 16, 8])
    learning_rate = trial.suggest_categorical(
        "learning_rate", [0.2, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    )
    use_queue = trial.suggest_categorical("use_queue", [False, True])
    epochs = trial.suggest_categorical("epochs", [10, 50, 100, 200])

    model = GraphConvModel(
        n_tasks=1,
        mode="classification",
        dropout=dropout,
        graph_conv_layers=graph_conv_layers,
        dense_layer_size=dense_layer_size,
        batch_size=batch_size,
        learning_rate=learning_rate,
        use_queue=use_queue,
    )

    eval_results = model.evaluate(
        test_dataset, [dc.metrics.Metric(dc.metrics.roc_auc_score)]
    )
    auc_score = eval_results["roc_auc_score"]
    return auc_score


In [12]:
sampler = TPESampler(seed=10)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2023-05-30 00:58:26,883] A new study created in memory with name: no-name-01d8f100-cab9-4a20-96a3-b415eba96b36


/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 16] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 8, 8, 4] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 128, 64, 32, 16] which is of type list.

/Users/alvaroromangomez/.local/sha

[I 2023-05-30 00:58:32,128] Trial 0 finished with value: 0.6885856079404467 and parameters: {'dropout': 0.5, 'graph_conv_layers': [256, 128, 64, 32, 16], 'dense_layer_size': 64, 'batch_size': 32, 'learning_rate': 0.0001, 'use_queue': False, 'epochs': 50}. Best is trial 0 with value: 0.6885856079404467.


/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 16] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 8, 8, 4] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 128, 64, 32, 16] which is of type list.

/Users/alvaroromangomez/.local/sha

[I 2023-05-30 00:58:37,424] Trial 1 finished with value: 0.6401985111662531 and parameters: {'dropout': 0.4, 'graph_conv_layers': [64, 64, 64, 64, 64, 64, 64], 'dense_layer_size': 128, 'batch_size': 32, 'learning_rate': 0.001, 'use_queue': True, 'epochs': 100}. Best is trial 0 with value: 0.6885856079404467.


...

[I 2023-05-30 01:06:31,189] Trial 97 finished with value: 0.5248138957816377 and parameters: {'dropout': 0.30000000000000004, 'graph_conv_layers': [128, 128, 64, 32, 32, 16, 16], 'dense_layer_size': 200, 'batch_size': 16, 'learning_rate': 0.1, 'use_queue': False, 'epochs': 200}. Best is trial 12 with value: 0.8635235732009926.


/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 16] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 8, 8, 4] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 128, 64, 32, 16] which is of type list.

/Users/alvaroromangomez/.local/sha

[I 2023-05-30 01:06:36,865] Trial 98 finished with value: 0.40694789081885857 and parameters: {'dropout': 0.30000000000000004, 'graph_conv_layers': [128, 128, 64, 32, 32, 16, 16], 'dense_layer_size': 64, 'batch_size': 16, 'learning_rate': 0.1, 'use_queue': False, 'epochs': 200}. Best is trial 12 with value: 0.8635235732009926.


/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 16] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [128, 128, 64, 32, 32, 16, 8, 8, 4] which is of type list.

/Users/alvaroromangomez/.local/share/virtualenvs/tfm_alvaro-vnpVahGc/lib/python3.10/site-packages/optuna/distributions.py:535: UserWarning:

Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256, 128, 64, 32, 16] which is of type list.

/Users/alvaroromangomez/.local/sha

[I 2023-05-30 01:06:42,211] Trial 99 finished with value: 0.7084367245657568 and parameters: {'dropout': 0.30000000000000004, 'graph_conv_layers': [64, 64, 64, 64, 64, 64, 64], 'dense_layer_size': 64, 'batch_size': 16, 'learning_rate': 0.01, 'use_queue': False, 'epochs': 10}. Best is trial 12 with value: 0.8635235732009926.


In [13]:
plot_optimization_history(study)

In [14]:
plot_param_importances(study)

In [15]:
best_model = GraphConvModel(
    n_tasks=1,
    mode="classification",
    dropout=study.best_params["dropout"],
    graph_conv_layers=study.best_params["graph_conv_layers"],
    dense_layer_size=study.best_params["dense_layer_size"],
    batch_size=study.best_params["batch_size"],
    learning_rate=study.best_params["learning_rate"],
    use_queue=study.best_params["use_queue"],
)

In [16]:
best_model.fit(train_dataset, nb_epoch=study.best_params["epochs"])

0.9526234436035156

In [17]:
best_model.evaluate(test_dataset, [dc.metrics.Metric(dc.metrics.roc_auc_score)])

{'roc_auc_score': 0.13523573200992556}